In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

In [2]:
import pome

# Generate example dummy data

In [3]:
np.random.seed(0)

# market data
if not os.path.exists("example_market.csv"):
    idx = pd.date_range("2018-04-01 00:00:00", "2018-06-30 23:59:59", freq="1S")
    ask = np.cumsum(np.random.rand(len(idx)) - 0.5) + 100.0
    bid = ask - 0.02
    pd.DataFrame(
        index=idx,
        data=np.array([ask, bid]).T,
        columns=["ask", "bid"]
    ).abs().to_csv("example_market.csv")

# signal data
if not os.path.exists("example_signal.csv"):
    sig = np.random.rand(3, len(idx)).T
    n = np.sum(sig, axis=1)
    pd.DataFrame(
        index=idx,
        data=sig / n.reshape(-1, 1),
        columns=["down", "neutral", "up"]
    ).to_csv("example_signal.csv")

# Configuration

In [4]:
model_id = ""
start_dt = "2018-06-03 00:00:00+0000"
end_dt = "2018-06-10 00:00:00+0000"
sig_url = "file:///{}/example_signal.csv".format(os.getcwd())
mkt_url = "file:///{}/example_market.csv".format(os.getcwd())

# loading the signal

In [5]:
from pome import signal

SYMBOL = "USDJPY"
sig = signal.load_signal(SYMBOL, sig_url, start_dt, end_dt)

In [6]:
sig.tail()

,down,neutral,up,argmax,pred
2018-06-09 23:59:56,0.134798,0.506867,0.358335,1,0.0
2018-06-09 23:59:57,0.553165,0.219549,0.227286,2,-1.0
2018-06-09 23:59:58,0.482775,0.218647,0.298578,2,-1.0
2018-06-09 23:59:59,0.094533,0.649231,0.256236,1,0.0
2018-06-10 00:00:00,0.486241,0.013714,0.500045,0,1.0


In [7]:
sig.symbol

'USDJPY'

# loading the marketdata

In [8]:
from pome import datasource

mkt = datasource.load_marketdata(
    sig.symbol,
    sig.start_dt,
    sig.end_dt,
    mkt_url,
)

In [9]:
mkt.tail()

,ask,bid
2018-06-09 23:59:56,66.810823,66.790823
2018-06-09 23:59:57,67.245930,67.225930
2018-06-09 23:59:58,67.228236,67.208236
2018-06-09 23:59:59,67.080989,67.060989
2018-06-10 00:00:00,66.930760,66.910760


# generating labels

In [10]:
from pome import labelizer


lbl = labelizer.generate_labels(
    mkt,
    labelizer.FixedNeutralLabelizer(
        lookahead="3Min",
        neutral_atol=0.075,
        neutral_rtol=0.00,
    ))

In [11]:
lbl.head()

,label_diff,label
2018-06-03 00:00:00,0.786592,1.0
2018-06-03 00:00:01,0.846372,1.0
2018-06-03 00:00:02,0.076975,1.0
2018-06-03 00:00:03,-0.245281,-1.0
2018-06-03 00:00:04,-0.237024,-1.0


In [12]:
lbl.label_type

'ternary'

# simulate trading strategy

In [13]:
from pome import strategies
trades = strategies.simple_buy_sell(mkt, sig, target_column_name="???")

In [14]:
trades.head()

,amount,ask,bid
2018-06-03 00:00:00,-1.0,143.016552,142.996552
2018-06-03 00:00:01,1.0,143.319539,143.299539
2018-06-03 00:00:02,-1.0,143.755250,143.735250
2018-06-03 00:00:03,0.0,143.703420,143.683420
2018-06-03 00:00:04,0.0,143.811219,143.791219


# calculate metrics

In [15]:
# from pome import metrics
# Not Implemented
# m_model = metrics.calc_metrics(sig, lbl)

# simulate and evaluate positions

In [16]:
from pome import positions
positions = positions.calc_positions(trades, mkt)

In [17]:
positions.head()

,ask,bid,amount,price
2018-06-03 00:00:00,143.016552,142.996552,-1.0,143.006552
2018-06-03 00:00:01,143.319539,143.299539,0.0,143.309539
2018-06-03 00:00:02,143.755250,143.735250,-1.0,143.745250
2018-06-03 00:00:03,143.703420,143.683420,-1.0,143.693420
2018-06-03 00:00:04,143.811219,143.791219,-1.0,143.801219


In [18]:
from pome import metrics
pl = metrics.evaluate_position(positions)

In [19]:
print("total_pl: {}".format(pl.total_pl))
print("averagel_pl: {}".format(pl.average_pl))
print("total_win: {}".format(pl.total_win))
print("total_loss: {}".format(pl.total_loss))

total_pl: 179671.21633114992
averagel_pl: 0.4455114813364756
total_win: 33121167.403502513
total_loss: -32941496.18717137


# simulate and evaluate portfolio
- Portfolio should be needed if we want to evaluate the performance of multi assets.
- e.g. When we evaluate USDJPY with ask/bid prices, we should have the positions of USDJPY and bank account(JPY).

In [20]:
# from pome import portfolio
# portfolio = portfolio.calc_positions(trades, mkt)

In [21]:
# portfolio{0} should be positions of USDJPY
# portfolio{1} should be positions of bank account(JPY).

In [22]:
# pl = metrics.evaluate_portfolio(portfolio)

# plot charts

In [23]:
from pome import plots
plots.historical_signals(mkt, sig, lbl)

ImportError: cannot import name 'plots'

In [ ]:
plots.historical_signals_with_trades(mkt, sig, lbl, trades)